In [1]:
!pip install --upgrade -q langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub

  Using cached chromadb-0.5.3-py3-none-any.whl (559 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Using cached fastapi-0.111.0-py3-none-any.whl (91 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl (62 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl (41 kB)
  Using cached onnxruntime-1.18.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl (59 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.25.0-py3-none-any.whl (18 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.46b0-py3-none-any.whl (11 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl (107 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv


In [3]:
load_dotenv()

# loading API key
os.getenv("#put your gemini api key here")

# service configuration
genai.configure(api_key=os.getenv("put your gemini api key here"))

In [4]:
def get_pdf_text(pdf_docs):
    """
    Extract and return the text data from a pdf file.

    Parameters:
    pdf_docs: It is the path of a pdf file.

    Returns:
    string: it will return the concatenated text data of the pdf file
    """
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

In [5]:
def get_text_chunks(text):

    # divide the text into 500 tokens chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=500)

    # splittinf based on the token chunks
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
# load the pdf file
pdf_docs = "/content/230205598.pdf"


# extracting, splitting, and creating embeddings
raw_text = get_pdf_text(pdf_docs)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)
# providing the input query
input_text = "Summarize the document in 5 line?"

# loading the embedding from the model
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

# vector similarity search of input query with generated FAISS index
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(input_text)

# creating a prompt template
template = """
Give a detailed explanation on\n\n
Context:\n {Textdata}?\n
Question: \n{askQuesion}\n

Answer:
"""

# initialize the google's gemini-Pro model for text analysis
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

prompt = PromptTemplate(template=template, input_variables=["Textdata", "askQuesion"])
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

# get the output response
response = chain(
    {"input_documents":docs, "question":input_text}
    , return_only_outputs=True)

IsADirectoryError: [Errno 21] Is a directory: '/'